In [1]:
%scala
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions.{trim,ltrim,rtrim,col}

Intitializing Scala interpreter ...

Spark Web UI available at http://f5b7b299c78b:4041
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1666220544835)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions.{trim, ltrim, rtrim, col}


In [4]:
val spark = SparkSession
 .builder
 .appName("Practica El Padron")
 .enableHiveSupport()
 .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@777a8a9a


In [5]:
val csvFile= "Rango_Edades_Seccion_202209.csv"

var dframe = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "DROPMALFORMED")
 .option("escapeQuotes", "true")
 .option("delimiter", ";")
 .load(csvFile)
 .toDF()

csvFile: String = Rango_Edades_Seccion_202209.csv
dframe: org.apache.spark.sql.DataFrame = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [6]:
dframe = dframe.withColumn("COD_DISTRITO",trim(col("COD_DISTRITO")))
.withColumn("COD_DIST_BARRIO",trim(col("COD_DIST_BARRIO")))
.withColumn("DESC_DISTRITO",trim(col("DESC_DISTRITO")))
.withColumn("DESC_BARRIO",trim(col("DESC_BARRIO")))
.withColumn("COD_BARRIO",trim(col("COD_BARRIO")))
.withColumn("COD_DIST_SECCION",trim(col("COD_DIST_SECCION")))
.withColumn("COD_SECCION",trim(col("COD_SECCION")))
.withColumn("COD_EDAD_INT",trim(col("COD_EDAD_INT")))
.withColumn("EspanolesHombres",trim(col("EspanolesHombres")))
.withColumn("EspanolesMujeres",trim(col("EspanolesMujeres")))
.withColumn("ExtranjerosHombres",trim(col("ExtranjerosHombres")))
.withColumn("ExtranjerosMujeres",trim(col("ExtranjerosMujeres")))

dframe: org.apache.spark.sql.DataFrame = [COD_DISTRITO: string, DESC_DISTRITO: string ... 10 more fields]


In [7]:
dframe = dframe.na.fill("0",Array("ExtranjerosMujeres"))
.na.fill("0",Array("ExtranjerosHombres"))
.na.fill("0",Array("EspanolesMujeres"))
.na.fill("0",Array("EspanolesHombres"))
.na.fill("0",Array("COD_EDAD_INT"))
.na.fill("0",Array("COD_SECCION"))
.na.fill("0",Array("COD_DIST_SECCION"))
.na.fill("0",Array("COD_BARRIO"))
.na.fill("0",Array("DESC_BARRIO"))
.na.fill("0",Array("COD_DIST_BARRIO"))
.na.fill("0",Array("DESC_DISTRITO"))
.na.fill("0",Array("COD_DISTRITO"))

dframe.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           0|               1|               1|                 0|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|          1|           1|               3|               3|                 1|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1001|      

dframe: org.apache.spark.sql.DataFrame = [COD_DISTRITO: string, DESC_DISTRITO: string ... 10 more fields]


In [8]:
dframe.select(countDistinct("COD_DISTRITO") as "Número de barrios diferentes").show()

+----------------------------+
|Número de barrios diferentes|
+----------------------------+
|                          21|
+----------------------------+



In [9]:
var tempView = dframe.createOrReplaceTempView("Padron")
spark.sql("select COUNT(DISTINCT COD_DISTRITO) from Padron").show()

+----------------------------+
|count(DISTINCT COD_DISTRITO)|
+----------------------------+
|                          21|
+----------------------------+



tempView: Unit = ()


In [10]:
dframe.select("DESC_DISTRITO")
.groupBy("DESC_DISTRITO")
.agg(length($"DESC_DISTRITO").alias("Longitud")).show()

+-------------------+--------+
|      DESC_DISTRITO|Longitud|
+-------------------+--------+
|             LATINA|       6|
|             TETUAN|       6|
|          SALAMANCA|       9|
|             RETIRO|       6|
|    MONCLOA-ARAVACA|      15|
|          HORTALEZA|       9|
| PUENTE DE VALLECAS|      18|
|         VILLAVERDE|      10|
|           CHAMBERI|       8|
|      CIUDAD LINEAL|      13|
|SAN BLAS-CANILLEJAS|      19|
|          VICALVARO|       9|
|        CARABANCHEL|      11|
|             CENTRO|       6|
|          CHAMARTIN|       9|
|          MORATALAZ|       9|
|FUENCARRAL-EL PARDO|      19|
|         ARGANZUELA|      10|
|              USERA|       5|
|  VILLA DE VALLECAS|      17|
+-------------------+--------+
only showing top 20 rows



In [11]:
val partition_dframe = dframe.write.partitionBy("DESC_DISTRITO","DESC_BARRIO")
dframe.rdd.partitions.size

partition_dframe: org.apache.spark.sql.DataFrameWriter[org.apache.spark.sql.Row] = org.apache.spark.sql.DataFrameWriter@58995ad8
res5: Int = 6


In [12]:
dframe.cache()

res6: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: string, DESC_DISTRITO: string ... 10 more fields]


In [13]:
dframe.groupBy("DESC_BARRIO","DESC_DISTRITO")
.agg(sum("EspanolesHombres").as("EspanolesHombres"),
    sum("EspanolesMujeres").as("EspanolesMujeres"),
    sum("ExtranjerosHombres").as("ExtranjerosHombres"),
    sum("ExtranjerosMujeres").as("ExtranjerosMujeres"),
    )
.orderBy(desc("ExtranjerosMujeres"))
.show()

+--------------------+-------------------+----------------+----------------+------------------+------------------+
|         DESC_BARRIO|      DESC_DISTRITO|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+--------------------+-------------------+----------------+----------------+------------------+------------------+
|           SAN DIEGO| PUENTE DE VALLECAS|         13960.0|         15692.0|            7477.0|            7700.0|
|              ALUCHE|             LATINA|         24734.0|         29534.0|            5856.0|            6801.0|
|        PUEBLO NUEVO|      CIUDAD LINEAL|         23044.0|         27100.0|            5980.0|            6793.0|
|        VISTA ALEGRE|        CARABANCHEL|         15818.0|         19307.0|            6000.0|            6648.0|
|            NUMANCIA| PUENTE DE VALLECAS|         16960.0|         19250.0|            6036.0|            6269.0|
|         EMBAJADORES|             CENTRO|         16266.0|         16138.0|    

In [14]:
dframe.unpersist()

res8: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: string, DESC_DISTRITO: string ... 10 more fields]


In [16]:
import org.apache.spark.sql.expressions.Window
val windowSpec  = Window.partitionBy("DESC_BARRIO","DESC_DISTRITO").orderBy("EspanolesHombres")

import org.apache.spark.sql.expressions.Window
windowSpec: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4f4deea6


In [29]:
val pivotDF = dframe.filter(dframe("DESC_DISTRITO") === "BARAJAS" or dframe("DESC_DISTRITO") === "CENTRO" or dframe("DESC_DISTRITO") === "RETIRO")
.groupBy("COD_EDAD_INT").pivot("DESC_DISTRITO").agg(sum("EspanolesMujeres"))
.show()

+------------+-------+------+------+
|COD_EDAD_INT|BARAJAS|CENTRO|RETIRO|
+------------+-------+------+------+
|          51|  439.0| 791.0| 811.0|
|           7|  218.0| 239.0| 475.0|
|          15|  270.0| 243.0| 401.0|
|          54|  346.0| 766.0| 797.0|
|         101|    7.0|  24.0|  17.0|
|          11|  261.0| 246.0| 425.0|
|          69|  246.0| 532.0| 836.0|
|          29|  179.0| 834.0| 657.0|
|          42|  353.0| 794.0| 704.0|
|         112|   null|   1.0|  null|
|          87|  107.0| 328.0| 470.0|
|          73|  284.0| 488.0| 829.0|
|          64|  212.0| 754.0| 838.0|
|           3|  165.0| 215.0| 363.0|
|          30|  200.0| 902.0| 623.0|
|         113|   null|  null|   1.0|
|          34|  215.0| 831.0| 569.0|
|          59|  309.0| 715.0| 861.0|
|           8|  228.0| 227.0| 400.0|
|          28|  156.0| 801.0| 560.0|
+------------+-------+------+------+
only showing top 20 rows



pivotDF: Unit = ()


In [22]:
dframe.write.option("header",true)
        .partitionBy("DESC_DISTRITO","DESC_BARRIO")
        .mode("overwrite")
        .csv("Prueba")

In [23]:
dframe.write.option("header",true)
        .partitionBy("DESC_DISTRITO","DESC_BARRIO")
        .mode("overwrite")
        .parquet("PruebaParquet")

In [24]:
//spark.sql("use default")
//spark.sql("SET hive.exec.dynamic.partition = true")
//spark.sql("create table if not exists default.DatosPadron(COD_DISTRITO String, DESC_DISTRITO String, COD_DIST_BARRIO String)")
//spark.sql("describe formatted default.DatosPadron")

org.apache.spark.sql.AnalysisException:  Hive support is required to CREATE Hive TABLE (AS SELECT);